In [21]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import Callback
from sklearn.model_selection import train_test_split

In [23]:
# Custom callback
class ProgressLogger(Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f"Epoch {epoch + 1}/{self.params['epochs']} - Progress logged")

In [24]:
# Load and fit the LabelEncoder on the full dataset
label_encoder = LabelEncoder()
y = pd.read_csv('/content/drive/MyDrive/Wail-Projet-F/CNN/Data/normalized_dataset.csv')['Best Algorithm'].values
label_encoder.fit(y)  # Fit on all unique labels in the original dataset
# Define algorithm names manually
algorithm_names = {
    0: 'BIPOP-CMA-ES',
    1: 'CMA-CSA_Atamna',
    2: 'CMAES-APOP-KMA_Nguyen',
    3: 'DE-BFGS_voglis_noiseless',
    4: 'a-CMA-ES',
    5: 'ad-CMA-ES_Gissler',
    6: 'adm-CMA-ES_Gissler',
    7: 'dm-CMA-ES_Gissler',
    8: 's-CMA-ES_Gissler',
    9: 'sd-CMA-ES_Gissler'
}
print(f"Algorithm names loaded: {algorithm_names}")

Algorithm names loaded: {0: 'BIPOP-CMA-ES', 1: 'CMA-CSA_Atamna', 2: 'CMAES-APOP-KMA_Nguyen', 3: 'DE-BFGS_voglis_noiseless', 4: 'a-CMA-ES', 5: 'ad-CMA-ES_Gissler', 6: 'adm-CMA-ES_Gissler', 7: 'dm-CMA-ES_Gissler', 8: 's-CMA-ES_Gissler', 9: 'sd-CMA-ES_Gissler'}


In [25]:
# -----------------------------------
# 1. Data Loading and Preprocessing
# -----------------------------------

# Load the dataset
data = pd.read_csv('/content/drive/MyDrive/Wail-Projet-F/CNN/Data/normalized_dataset.csv')

# Investigate row count
unique_combinations = data[['Function', 'Instance', 'Dimension']].drop_duplicates()

# Define columns to drop
columns_to_drop = ['FID', 'IID', 'Dimension', 'ERT', 'min_ERT', 'RELERT']

# Drop specified columns if they exist
data = data.drop(columns=[col for col in columns_to_drop if col in data.columns], axis=1)

# Define features and target
X = data.drop('Best Algorithm', axis=1).values
y = data['Best Algorithm'].values

# Check initial shapes
print(f"Original X shape: {X.shape}")
print(f"Original y shape: {y.shape}")

# Check class distribution before SMOTE
print("\nClass Distribution before SMOTE:")
class_counts = pd.Series(y).value_counts().sort_index()
print(class_counts)

# Determine minimum class size for SMOTE k_neighbors
min_class_size = class_counts.min()
k_neighbors = min(3, max(1, min_class_size - 1))

# Apply SMOTE to balance classes
smote = SMOTE(sampling_strategy='not majority', k_neighbors=k_neighbors, random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Check resampled shapes
print(f"Resampled X shape before reshape: {X_resampled.shape}")
print(f"Resampled y shape: {y_resampled.shape}")

# Convert labels to categorical
y_resampled = label_encoder.transform(y_resampled)
y_resampled = tf.keras.utils.to_categorical(y_resampled, num_classes=len(algorithm_names))

# Reshape data for CNN
n_features = X_resampled.shape[1]  # Use the feature count after SMOTE
print(f"Number of features in X_resampled: {n_features}")
side_length = int(np.ceil(np.sqrt(n_features)))
padding = side_length ** 2 - n_features
X_resampled = np.pad(X_resampled, ((0, 0), (0, padding)), mode='constant')  # Pad each sample
print(f"X_resampled shape after padding: {X_resampled.shape}")
X_resampled = X_resampled.reshape(-1, side_length, side_length, 1)
print(f"Reshaped X_resampled shape: {X_resampled.shape}")

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, shuffle=True)

# Print class distributions
print("\nClass distribution in y_train:", np.sum(y_train, axis=0))
print("Class distribution in y_test:", np.sum(y_test, axis=0))

Original X shape: (480, 291)
Original y shape: (480,)

Class Distribution before SMOTE:
0     27
1     41
2     26
3    193
4     48
5     36
6     31
7     31
8     17
9     30
Name: count, dtype: int64
Resampled X shape before reshape: (1930, 291)
Resampled y shape: (1930,)
Number of features in X_resampled: 291
X_resampled shape after padding: (1930, 324)
Reshaped X_resampled shape: (1930, 18, 18, 1)

Class distribution in y_train: [150. 156. 160. 154. 160. 157. 152. 156. 154. 145.]
Class distribution in y_test: [43. 37. 33. 39. 33. 36. 41. 37. 39. 48.]


In [26]:
# -----------------------------------
# 2. Model Definition
# -----------------------------------

# Define the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(side_length, side_length, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(algorithm_names), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [27]:
# -----------------------------------
# 3. Model Training
# -----------------------------------

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)])

# Calculate and print final training accuracy and loss
train_loss, train_accuracy = model.evaluate(X_train, y_train, verbose=0)
print(f"\nFinal Training Accuracy: {train_accuracy:.4f}")
print(f"Final Training Loss: {train_loss:.4f}")

# Save results to file
with open('/content/drive/MyDrive/Wail-Projet-F/CNN/Result/result.txt', 'w') as f:
    f.write(f"Final Training Accuracy: {train_accuracy:.4f}\n")
    f.write(f"Final Training Loss: {train_loss:.4f}\n")

Epoch 1/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.0835 - loss: 2.3060 - val_accuracy: 0.1392 - val_loss: 2.3003
Epoch 2/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.1375 - loss: 2.2948 - val_accuracy: 0.1812 - val_loss: 2.2794
Epoch 3/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.1668 - loss: 2.2627 - val_accuracy: 0.1165 - val_loss: 2.2763
Epoch 4/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.1413 - loss: 2.2731 - val_accuracy: 0.2201 - val_loss: 2.2251
Epoch 5/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.1697 - loss: 2.2397 - val_accuracy: 0.2265 - val_loss: 2.1974
Epoch 6/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.1816 - loss: 2.2227 - val_accuracy: 0.2233 - val_loss: 2.1545
Epoch 7/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.2169 - loss: 2.1723 - val_accuracy: 0.3010 - val_loss: 2.0953
Epoch 8/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.2171 - loss: 2.1231 - val_accuracy: 0.

In [28]:
# -----------------------------------
# 4. Model Visualization and Saving
# -----------------------------------

# Plot training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5), dpi=100)

# Accuracy plot
ax1.plot(history.history['accuracy'], label='Training Accuracy')
ax1.plot(history.history['val_accuracy'], label='Validation Accuracy')
ax1.set_title('Model Accuracy')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.set_ylim(0.0, 1.0)
ax1.legend(loc='lower right')
ax1.grid(True)

# Loss plot
ax2.plot(history.history['loss'], label='Training Loss')
ax2.plot(history.history['val_loss'], label='Validation Loss')
ax2.set_title('Model Loss')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.set_ylim(0.0, max(history.history['loss']) * 1.1)
ax2.legend(loc='upper right')
ax2.grid(True)

plt.tight_layout()
plt.savefig('/content/drive/MyDrive/Wail-Projet-F/CNN/Result/model_architecture.png')
print(f"Accuracy and Loss plot saved to: /content/drive/MyDrive/Wail-Projet-F/CNN/Result/model_architecture.png")
plt.close()

Accuracy and Loss plot saved to: /content/drive/MyDrive/Wail-Projet-F/CNN/Result/model_architecture.png


In [29]:
# -----------------------------------
# 5. Model Evaluation on Test Set
# -----------------------------------

# Evaluate on test set
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f'Test Accuracy: {test_acc:.3f}')
print(f'Test Loss: {test_loss:.3f}')

# Append test results to file
with open('/content/drive/MyDrive/Wail-Projet-F/CNN/Result/result.txt', 'a') as f:
    f.write(f"Test Accuracy: {test_acc:.3f}\n")
    f.write(f"Test Loss: {test_loss:.3f}\n")

# Generate predictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Compute confusion matrix
cm = confusion_matrix(y_test_classes, y_pred_classes)

# Convert to percentages
cm_percentage = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100

# Plot confusion matrix with algorithm names
fig, ax = plt.subplots(figsize=(12, 12), dpi=100)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_percentage, display_labels=[algorithm_names[i] for i in range(len(algorithm_names))])
disp.plot(cmap=plt.cm.Blues, ax=ax, values_format=".2f")
plt.title('Confusion Matrix with Percentages', pad=20)
plt.xlabel('Predicted Algorithm')
plt.ylabel('True Algorithm')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig('/content/drive/MyDrive/Wail-Projet-F/CNN/Result/confusion_matrix.png')
print(f"Confusion Matrix plot saved to: /content/drive/MyDrive/Wail-Projet-F/CNN/Result/confusion_matrix.png")
plt.close()

# Print classification report
print("\nClassification Report:")
print(classification_report(y_test_classes, y_pred_classes, zero_division=0, target_names=[algorithm_names[i] for i in range(len(algorithm_names))]))

# Append classification report to file
with open('/content/drive/MyDrive/Wail-Projet-F/CNN/Result/result.txt', 'a') as f:
    f.write("\nClassification Report:\n")
    f.write(classification_report(y_test_classes, y_pred_classes, zero_division=0, target_names=[algorithm_names[i] for i in range(len(algorithm_names))]))

# Save the model
model.save('/content/drive/MyDrive/Wail-Projet-F/CNN/Result/cnn_model.h5')
print("Model saved to: /content/drive/MyDrive/Wail-Projet-F/CNN/Result/cnn_model.h5")

Test Accuracy: 0.886
Test Loss: 0.771
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


Confusion Matrix plot saved to: /content/drive/MyDrive/Wail-Projet-F/CNN/Result/confusion_matrix.png

Classification Report:
                          precision    recall  f1-score   support

            BIPOP-CMA-ES       0.93      1.00      0.97        43
          CMA-CSA_Atamna       0.94      0.81      0.87        37
   CMAES-APOP-KMA_Nguyen       0.84      0.82      0.83        33
DE-BFGS_voglis_noiseless       0.95      0.54      0.69        39
                a-CMA-ES       0.87      0.79      0.83        33
       ad-CMA-ES_Gissler       0.76      0.94      0.84        36
      adm-CMA-ES_Gissler       0.87      0.98      0.92        41
       dm-CMA-ES_Gissler       0.80      1.00      0.89        37
        s-CMA-ES_Gissler       0.97      0.97      0.97        39
       sd-CMA-ES_Gissler       0.96      0.96      0.96        48

                accuracy                           0.89       386
               macro avg       0.89      0.88      0.88       386
            wei